# 03 Scan surface location optimizing the quasi-symmetry

Prepared by: Caoxiang Zhu for Group 3 at the Summer School on Stellarator Optimization 2020

## Introduction

STELLOPT can optimize the value of quasi-symmetry on an arbitrary winding surface.
One a [VMEC](https://princetonuniversity.github.io/STELLOPT/VMEC) equilibrium is calculated, STELLOPT will call [BOOZ_XFORM](https://princetonuniversity.github.io/STELLOPT/BOOZ_XFORM) to convert VMEC coordinates into Boozer coordinates.
Afterward, Fourier decomposition of $|\vec{B}|$ in Boozer coordinates will be performed,
$$
|\vec{B}| = \sum_{n,m} B_{mn} \cos(m\theta - n\zeta).
$$
Here, stellarator symmetry is assumed and only even terms are used.
To improve quasi-symmetry, STELLOPT will try to reduce the non-symmetric terms,
$$
f_{QS} = \sum_{n/m \neq N/M} {B_{mn}}^2 ,
$$
where $N, M$ are the symmetry indicator.

There are three types of quasi-symmetry: 
  - Quasi-axisymmetry (QA): $N=0$. The magnetic field strength is axisymmetric (toroidally).
  - Quasi-poloidal symmetry (QP): $M=0$. The magnetic field strength is poloidally symmetric.
  - Quasi-helical symmetry (QH): $N, M \neq 0$. The magnetic field strength is helically symmetric.
 
In the STELLOPT input namelist [optim](https://princetonuniversity.github.io/STELLOPT/STELLOPT%20Input%20Namelist), we can control the target of quasi-symmetry.
For example, the following snippet means we want to optimize QA on the VMEC flux surface labeled from 2 to 49 over 49 surfaces (index 1 stands for the magnetic axis).
```
!------------------------------------------------------------------------
!       Boozer Coordinate Helicity
!         Note that helicity targeting is by surface.  Axis (01) is ignored.
!         (X,0): Quasi-Axisymetry
!         (0,X): Quasi-Poloidal Symmetry
!         (L,K): Quasi-Helical Symmetry (m *K - n*L)
!------------------------------------------------------------------------
  HELICITY = (1,0)
  TARGET_HELICITY(2:49) = 48*0.0  SIGMA_HELICITY(2:49) = 48*1.0
```

## Varying the target surface to optimize quasi-symmetry

There is a freedom in choosing the target surface on which the quasi-symmetry is improved.
Historically, multiple surfaces were used from the core to the edge.
[Henneberg et al.](https://iopscience.iop.org/article/10.1088/1361-6587/ab5add) reported it that varying the position of the target flux surface will have different results. 
Here we are going to briefly explore the effect of the target surface position on the performance of quasi-symmetry.

We will start with an optimization that tries to optimize all the flux surfaces in the equilibrium, then scan optimizing the single surface at s=0.25, 0.50, 0.75, 1.00 (s is the normalized toroidal flux).

## Optimize all the surfaces

### Check the input

You can find the input file in the `./all` folder.
It has a `input.all` file used for STELLOPT.
You can open it using your preferred editor, like `vi input.all`, `emacs input.all`, `more input.all`, etc.

Question: Can you explain what the input file is doing? What are the free parameters? What are the targets? Which optimization algorithm is it using? ...

### Run STELLOPT

Do the following in the terminal

```shell
mpiexec -n 8 xstelloptv2 input.all
```

There are in total 24 CPUs for each group.
You can discuss with your group members and determine how many CPUs you want to use for each job.

### Run BOOZ_XFORM for post-processing

You can now run BOOZ_XFORM to calculate the Fourier coefficients of $|\vec{B}|$ in Boozer coordinates.
The input file is `./inbooz.reset_file` and you can check the details of the input file.
We are going to convert the VMEC output `wout_reset_file.nc`, which stands for the most recent VMEC calculation during the iterations.
```
xbooz_xform ../inbooz.reset_file
```

### Check STELLOPT results

Now we can check the STELLOPT results.
The main output is `stellopt.all` file in which STELLOPT writes the status of each iteration.
We can read this file by using the python code [`coilpy.STELLout` ](https://github.com/zhucaoxiang/CoilPy/blob/master/coilpy/stellopt.py#L57), or something else you like.

In [ ]:
# Update to the latest version of coilpy in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -U coilpy

In [1]:
# import required packages
import matplotlib.pyplot as plt
import numpy as np
from coilpy import STELLout

In [ ]:
# read the ooutput
stell_all = STELLout('stellopt.all')

In [ ]:
# plot the convergence
stell_all.plot()

In [ ]:
# plot the evolution of the QA value
f, ax = plt.subplots()
for i in range(len(stell_all['ITER'])):
    stell_all.plot_helicity(it=i, ax=ax, label='iter={:03d}'.format(stell_all['ITER'][i]))

### Check the VMEC equilibrium

For each successiful iteration, STELLOPT will save the VMEC output (and the results from other codes as well).
You can check the status of VMEC equilibrium at each iteration.
```
ls -al wout_*
```

In [ ]:
# Read VMEC output file
from coilpy import VMECout
init = VMECout('wout_full.00000.nc')

In [ ]:
# Check the flux surface shape
init.plot('LPK')

In [ ]:
# Check iota profile
init.plot('Pressure')

In [ ]:
# Check Pressure profile
init.plot('Pressure')

In [ ]:
# Read the last equilibrium
final = VMECout('wout_full.reset_file.nc')

In [ ]:
# Compare the shape of flux surfaces
plt.figure()
init.surface[-1].plot(zeta=0, color='b', linestyle='--')
init.surface[-1].plot(zeta=np.pi/3, color='b', linestyle='--')
final.surface[-1].plot(zeta=0, color='r', linestyle='-')
final.surface[-1].plot(zeta=np.pi/3, color='r', linestyle='-')

### Check BOOZ_XFORM results

We can also check the BOOZ_XFORM results in the file `boozmn.reset_file.nc`.

In [ ]:
# read the output
from coilpy import BOOZ_XFORM
booz = BOOZ_XFORM('boozmn_reset_file.nc')

In [ ]:
# plot non-axisymmetric terms
booz.plot()

In [ ]:
# plot |B| contour on one surface
booz.plot2d(ns=24)

Now please run the other four cases and compare the optimization results.